In [218]:
######ITERATION 4 CODE######


import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('INFOSYS_722_IT4_Jack_Whooley').getOrCreate()

###

hd1 = spark.read.csv("./heartdisease1.csv",  header="true" )

###

hd2 = spark.read.csv("./heartdisease2.csv",  header="true" )

###








In [224]:
#appending hd1 to hd2

hd = hd1.union(hd2)

hd.describe().show()
##



+-------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|summary|              age|               sex|     chestpaintype|       restingbps|       cholesterol| fastingbloodsugar|        restingecg|      maxheartrate|    exerciseangina|           oldpeak|           STslope|             target|
+-------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|  count|             1190|              1189|              1189|             1189|              1189|              1190|              1190|              1190|              1190|              1190|              1190|               1190|
|   mean|53.72016806722689|0.7636669470142977|3.2321

In [225]:
#Creating two new variables that I hypothesise could have predictive power. I have chained 

from pyspark.sql.functions import col
import numpy as np
from pyspark.sql.functions import log

hdaddvar = hd.withColumn("maxHRrestingHR",  col("maxheartrate") * col("restingbps")).withColumn("MaxHRLogAge", col("maxheartrate") * log(col("age")))


hd.withColumn("sqrtcholesterol",   pow(col("cholesterol",0.5))

##



In [226]:
#Creating discrete age categories out of continuous age variable

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import when

              
hdnewer = hdaddvar.withColumn("young", (when(col("age") < 35, 1).otherwise(0))).withColumn("middleaged", (when(col("age") < 35, 0).when(col("age") > 65, 0).otherwise(1))).withColumn("old", (when(col("age") > 65, 1).otherwise(0)))

##



In [227]:
hd3 = hdnewer.na.drop()

hd3.describe().show()

+-------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-----------------+------------------+-------------------+------------------+-------------------+
|summary|              age|               sex|     chestpaintype|       restingbps|       cholesterol| fastingbloodsugar|        restingecg|      maxheartrate|     exerciseangina|           oldpeak|           STslope|            target|   maxHRrestingHR|       MaxHRLogAge|              young|        middleaged|                old|
+-------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-----------------+------------------+-------------------+------------------+-------------------+
|

In [228]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns: "label" and "features".

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler





In [229]:
hd3.columns

['age',
 'sex',
 'chestpaintype',
 'restingbps',
 'cholesterol',
 'fastingbloodsugar',
 'restingecg',
 'maxheartrate',
 'exerciseangina',
 'oldpeak',
 'STslope',
 'target',
 'maxHRrestingHR',
 'MaxHRLogAge',
 'young',
 'middleaged',
 'old']

In [230]:
# Combine all features into one vector named features.
assembler = VectorAssembler(
  inputCols=['age',
 'sex',
 'chestpaintype',
 'restingbps',
 'cholesterol',
 'fastingbloodsugar',
 'restingecg',
 'maxheartrate',
 'exerciseangina',
 'oldpeak',
 'STslope',
 'maxHRrestingHR',
 'MaxHRLogAge',
 'young',
 'middleaged',
 'old'],
              outputCol="features")

In [231]:
from pyspark.sql.types import IntegerType
hd3 = hd3.withColumn("age", hd3["age"].cast(IntegerType()))
hd3 = hd3.withColumn("sex", hd3["sex"].cast(IntegerType()))
hd3 = hd3.withColumn("chestpaintype", hd3["chestpaintype"].cast(IntegerType()))
hd3 = hd3.withColumn("restingbps", hd3["restingbps"].cast(IntegerType()))
hd3 = hd3.withColumn("cholesterol", hd3["cholesterol"].cast(IntegerType()))
hd3 = hd3.withColumn("fastingbloodsugar", hd3["fastingbloodsugar"].cast(IntegerType()))
hd3 = hd3.withColumn("restingecg", hd3["restingecg"].cast(IntegerType()))
hd3 = hd3.withColumn("maxheartrate", hd3["maxheartrate"].cast(IntegerType()))
hd3 = hd3.withColumn("exerciseangina", hd3["exerciseangina"].cast(IntegerType()))
hd3 = hd3.withColumn("oldpeak", hd3["oldpeak"].cast(IntegerType()))
hd3 = hd3.withColumn("STslope", hd3["STslope"].cast(IntegerType()))
hd3 = hd3.withColumn("target", hd3["target"].cast(IntegerType()))
hd3 = hd3.withColumn("maxHRrestingHR", hd3["maxHRrestingHR"].cast(IntegerType()))
hd3 = hd3.withColumn("MaxHRLogAge", hd3["MaxHRLogAge"].cast(IntegerType()))

hd3.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- chestpaintype: integer (nullable = true)
 |-- restingbps: integer (nullable = true)
 |-- cholesterol: integer (nullable = true)
 |-- fastingbloodsugar: integer (nullable = true)
 |-- restingecg: integer (nullable = true)
 |-- maxheartrate: integer (nullable = true)
 |-- exerciseangina: integer (nullable = true)
 |-- oldpeak: integer (nullable = true)
 |-- STslope: integer (nullable = true)
 |-- target: integer (nullable = true)
 |-- maxHRrestingHR: integer (nullable = true)
 |-- MaxHRLogAge: integer (nullable = true)
 |-- young: integer (nullable = false)
 |-- middleaged: integer (nullable = false)
 |-- old: integer (nullable = false)



In [232]:
hd3.toPandas().to_csv(r"C:\Users\jackw\OneDrive\Documents\University 2020\Semester 2\INFOSYS 722\Iteration4_BDAS\hd44.csv")

In [257]:

# Let's transform the data. 
output = assembler.transform(hd3)

In [258]:
output.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- chestpaintype: integer (nullable = true)
 |-- restingbps: integer (nullable = true)
 |-- cholesterol: integer (nullable = true)
 |-- fastingbloodsugar: integer (nullable = true)
 |-- restingecg: integer (nullable = true)
 |-- maxheartrate: integer (nullable = true)
 |-- exerciseangina: integer (nullable = true)
 |-- oldpeak: integer (nullable = true)
 |-- STslope: integer (nullable = true)
 |-- target: integer (nullable = true)
 |-- maxHRrestingHR: integer (nullable = true)
 |-- MaxHRLogAge: integer (nullable = true)
 |-- young: integer (nullable = false)
 |-- middleaged: integer (nullable = false)
 |-- old: integer (nullable = false)
 |-- features: vector (nullable = true)



In [259]:
###Outlier Removal (Python Code)##

##removing null values, calculating z-scores

#hd.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

#hd = hd.select_dtypes(exclude=['object'])

#from scipy import stats
#hd['zage']=np.abs(stats.zscore(hd.age))
#hd['zrestingbps']=np.abs(stats.zscore(hd.restingbps))
#hd['zcholesterol']=np.abs(stats.zscore(hd.cholesterol))
#hd['zmaxheartrate']=np.abs(stats.zscore(hd.maxheartrate))
#hd['zoldpeak']=np.abs(stats.zscore(hd.oldpeak))
#hd['zSTslope']=np.abs(stats.zscore(hd.STslope))

############################################################

#removing if any z-score is greater than 3#

#hdnew = hd.drop(hd[(hd['zage'] > 3.0) | (hd['zrestingbps'] > 3.0) | (hd['zcholesterol'] > 3.0) | (hd['zmaxheartrate'] > 3.0) | (hd['zoldpeak'] > 3.0) | (hd['zSTslope'] > 3.0)].index)


#hdnew2 = hdnew.drop(['zage', 'zrestingbps', 'zcholesterol', 'zmaxheartrate', 'zoldpeak', 'zSTslope', 'Unnamed: 0'], axis=1)



In [260]:
##Dimensionality Reduction (Python Code)##

#from sklearn.feature_selection import VarianceThreshold

#sel = VarianceThreshold(threshold=(.8 * (1 - .8)))

#sel.fit_transform(hdnew2)



In [261]:
# Split the training and testing set.
train_data,test_data = output.randomSplit([0.8,0.2])

In [262]:
# Let's import the relevant classifiers. 
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline

In [263]:
# Use defaults to make the comparison "fair". This simplifies the comparison process.

dtc = DecisionTreeClassifier(labelCol='target',featuresCol='features')
rfc = RandomForestClassifier(labelCol='target',featuresCol='features', maxBins = 50, numTrees = 5, maxDepth = 5)
gbt = GBTClassifier(labelCol='target',featuresCol='features')

In [264]:
# Train the models (it's three models, so it might take some time).
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [265]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

In [266]:
# Let's start off with binary classification.
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Note that the label column isn't named label, it's named PrivateIndex in this case.
my_binary_eval = BinaryClassificationEvaluator(labelCol = 'target')

In [268]:
# This is the area under the curve. This indicates that the data is highly seperable.
print("DTC")
print(my_binary_eval.evaluate(dtc_predictions))

# RFC improves accuracy but also model complexity. RFC outperforms DTC in nearly every situation.
print("RFC")
print(my_binary_eval.evaluate(rfc_predictions))



DTC
0.8688974442988204
RFC
0.927383682830931


In [244]:
# Let's import the evaluator.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [245]:
# Select (prediction, true label) and compute test error. 
acc_evaluator = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="accuracy")

In [246]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [247]:
# Let's do something a bit more complex in terms of printing, just so it's formatted nicer. 
print("Here are the results!")
print('-'*40)
print('A single decision tree has an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*40)
print('A random forest ensemble has an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*40)
print('An ensemble using GBT has an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
----------------------------------------
A single decision tree has an accuracy of: 84.87%
----------------------------------------
A random forest ensemble has an accuracy of: 85.15%
----------------------------------------
An ensemble using GBT has an accuracy of: 88.80%


In [248]:
rfc_model.featureImportances

SparseVector(16, {0: 0.0734, 1: 0.027, 2: 0.2071, 3: 0.0323, 4: 0.0668, 5: 0.0161, 6: 0.0023, 7: 0.148, 8: 0.0364, 9: 0.1298, 10: 0.1805, 11: 0.0382, 12: 0.0339, 14: 0.004, 15: 0.0042})